<a href="https://colab.research.google.com/github/Marston/quant-apprentice/blob/main/the_quants_apprentice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quant Apprentice

⚙️ Agentic Workflow Patterns
Quant Apprentice implements three distinct workflow patterns that enable its complex, end-to-end analysis capabilities.

1. Prompt Chaining (Sequential Task Processing)

  This pattern is used for structured data processing tasks, particularly for news analysis. The agent executes a series of prompts in a specific order to transform raw data into a concise summary.

  Workflow: `Ingest News → Preprocess Text → Classify Sentiment → Extract Key Entities → Summarize Findings`

2. Routing (Specialized Sub-Agents)

  To handle diverse types of information, `the main agent routes tasks` to specialized sub-agents. Each sub-agent is an expert in a specific domain, ensuring that the right logic is applied to the right data.

  - Earnings Analyzer: Focuses on parsing quarterly earnings reports and financial statements.

  - News Analyzer: Specializes in sentiment analysis and summarization of market news.

  - Market Analyzer: Analyzes price action, volume, and technical indicators.

3. Evaluator–Optimizer Loop (Iterative Refinement)

  This is the agent's self-improvement mechanism. The system generates an initial analysis, evaluates it against a set of quality criteria, and then uses the feedback to generate a refined, more comprehensive final output.

  Workflow: `Generate Initial Analysis → Evaluate Quality & Completeness → Provide Feedback to Agent → Generate Refined Analysis`




In [1]:
# Clones the project repository from GitHub
!git clone https://github.com/marston/Quant-Apprentice.git

!git config --global user.email "shejo284@gmail.com"
!git config --global user.name "Marston Ward"
!git remote add origin https://github.com/marston/Quant-Apprentice.git

%cd Quant-Apprentice/

# Add the requirements.txt file to the staging area
!git fetch origin main
!git checkout main
print("Installing packages...")
%pip install --upgrade --quiet -r requirements.txt

fatal: destination path 'Quant-Apprentice' already exists and is not an empty directory.
fatal: not a git repository (or any of the parent directories): .git
/content/Quant-Apprentice
From https://github.com/marston/Quant-Apprentice
 * branch            main       -> FETCH_HEAD
Already on 'main'
Your branch is up to date with 'origin/main'.
Installing packages...


In [ ]:
# Commit the changes with a message
#!git commit -m "Add requirements.txt"
# Push the changes to the remote repository
#!git push origin main

In [2]:
# Step 2: Import necessary libraries and set up API key
import os
import yfinance as yf

from google.colab import userdata
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import TypedDict, List
from langgraph.graph import StateGraph, END

print("✅ Libraries installed.")

✅ Libraries installed.


In [ ]:
# List all the Models available. Use this when need to change the LLM

!curl "https://generativelanguage.googleapis.com/v1beta/models?key=API_KEY"

In [3]:
# Constants
LLM_MODEL = "gemini-2.5-flash"

# Load your key from Colab Secrets
try:
    api_key = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = api_key
    genai.configure(api_key=api_key)
    print("Google API Key configured successfully!")
except userdata.SecretNotFoundError:
    print("ERROR: 'GOOGLE_API_KEY' not found in Colab Secrets. Please add it.")
except Exception as e:
    print(f"An error occurred: {e}")


# Initialize the Gemini model we'll use for all patterns
# Using Google Flash
llm = ChatGoogleGenerativeAI(model=LLM_MODEL)

print("LLM Initialized. You are ready to build the agent patterns!")

Google API Key configured successfully!
LLM Initialized. You are ready to build the agent patterns!


In [4]:
print("--- ⚙️ Pattern 1: Prompt Chaining Workflow ---")

# --- Define Each Step (Prompt) in our Chain ---

# Step 1: Classify the sentiment of the article
sentiment_prompt = ChatPromptTemplate.from_template(
    "Analyze the following news article and classify its sentiment as strictly 'Positive', 'Negative', or 'Neutral'.\n\nArticle: {article}"
)

# Step 2: Extract key companies and people mentioned
entities_prompt = ChatPromptTemplate.from_template(
    "From this article, extract the key entities (Companies, People, Products) mentioned. Format them as a simple comma-separated list.\n\nArticle: {article}"
)

# Step 3: Summarize the article for a financial analyst
summary_prompt = ChatPromptTemplate.from_template(
    "You are a financial analyst providing a quick brief. Based on the article which has a '{sentiment}' sentiment, write a 2-sentence summary highlighting the key financial impact.\n\nArticle: {article}"
)

# --- Create the Chains for Each Step ---
# The '|' (pipe) operator is like a conveyor belt, linking the prompt, the model, and the output parser.
sentiment_chain = sentiment_prompt | llm | StrOutputParser()
entities_chain = entities_prompt | llm | StrOutputParser()
summary_chain = summary_prompt | llm | StrOutputParser()

# --- Run the Full Workflow with a sample article ---
news_article = """
NEW YORK – Shares of QuantumLeap Inc. surged over 25% today after the company unveiled its groundbreaking 'Fusion' processor.
CEO Dr. Evelyn Reed claimed the chip is 50 times faster than anything on the market, positioning them to dominate the data center industry.
Major cloud providers are already in talks for multi-billion dollar contracts.
"""

print("\n📰 Processing news article...")

# Execute Step 1
article_sentiment = sentiment_chain.invoke({"article": news_article})

# Execute Step 2
key_entities = entities_chain.invoke({"article": news_article})

# Execute Step 3, feeding in the sentiment from the previous step
final_summary = summary_chain.invoke({"sentiment": article_sentiment, "article": news_article})

# --- Display the structured results ---
print("\n--- ✅ Analysis Complete ---")
print(f"Sentiment: {article_sentiment}")
print(f"Key Entities: {key_entities}")
print(f"Final Summary: {final_summary}")

--- ⚙️ Pattern 1: Prompt Chaining Workflow ---

📰 Processing news article...

--- ✅ Analysis Complete ---
Sentiment: Positive
Key Entities: QuantumLeap Inc., Dr. Evelyn Reed, Fusion
Final Summary: QuantumLeap Inc. shares surged over 25% today following the unveiling of its groundbreaking 'Fusion' processor, indicating strong immediate investor confidence. This innovation is expected to secure multi-billion dollar contracts with major cloud providers, projecting substantial future revenue and market dominance.


In [5]:
print("--- ⚙️ Pattern 2: Routing Workflow ---")

# --- 1. Define Your Specialist Sub-Agents (as Python functions) ---

def earnings_analyzer(ticker: str):
    """
    Placeholder for a function that would analyze SEC filings or earnings call transcripts.
    """
    print(f" -> Routed to: Earnings Analyzer for {ticker}")
    return f"Analysis of {ticker}'s latest earnings report shows a 15% year-over-year revenue increase."

def news_analyzer(ticker: str):
    """
    Placeholder that could use our prompt chain from Pattern 1.
    """
    print(f"  -> Routed to: News Analyzer for {ticker}")
    return f"Analysis of recent news for {ticker} indicates positive sentiment following new product announcements."

def market_analyzer(ticker: str):
    """
    A simple market data analyzer that uses a real tool (yfinance).
    """
    print(f"  -> Routed to: Market Analyzer for {ticker}")
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="1mo")
        last_close = hist['Close'].iloc[-1]
        return f"Market data for {ticker}: The last closing price was ${last_close:.2f}."
    except Exception:
        return f"Could not fetch market data for {ticker}."

# --- 2. Create the Routing Logic ---
# This prompt asks the LLM to act as the dispatcher.
routing_template = """Given the user's query about a stock, classify it as one of the following categories: 'Earnings', 'News', or 'Market'. Respond with only a single word.

Query: {query}
Classification:"""

router_prompt = PromptTemplate.from_template(routing_template)
router_chain = router_prompt | llm | StrOutputParser()

# --- 3. Build the Main Router Function ---
def route_query(query: str, ticker: str):
    """Routes a query to the correct specialist function."""
    print(f"\nUser Query: '{query}' for {ticker}")
    destination = router_chain.invoke({"query": query}).strip()
    print(f" -> Routed to: {destination}")

    if "Earnings" in destination:
        return earnings_analyzer(ticker)
    elif "News" in destination:
        return news_analyzer(ticker)
    elif "Market" in destination:
        return market_analyzer(ticker)
    else:
        return f"Could not route query. Unknown destination: '{destination}'"

# --- 4. Test the Router ---
print("--- Testing Router with Different Queries ---")
print(f"Agent Response: {route_query('How did their profits look last quarter?', 'MSFT')}")
print(f"Agent Response: {route_query('What is the latest chatter about them?', 'NVDA')}")
print(f"Agent Response: {route_query('What is the current stock price?', 'AAPL')}")

--- ⚙️ Pattern 2: Routing Workflow ---
--- Testing Router with Different Queries ---

User Query: 'How did their profits look last quarter?' for MSFT
 -> Routed to: Earnings
 -> Routed to: Earnings Analyzer for MSFT
Agent Response: Analysis of MSFT's latest earnings report shows a 15% year-over-year revenue increase.

User Query: 'What is the latest chatter about them?' for NVDA
 -> Routed to: News
  -> Routed to: News Analyzer for NVDA
Agent Response: Analysis of recent news for NVDA indicates positive sentiment following new product announcements.

User Query: 'What is the current stock price?' for AAPL
 -> Routed to: Market
  -> Routed to: Market Analyzer for AAPL
Agent Response: Market data for AAPL: The last closing price was $254.43.


In [6]:
print("--- ⚙️ Pattern 3: Evaluator-Optimizer Loop with LangGraph ---")

# --- 1. Define the State ---
# This object is the "memory" that gets passed between the steps in our graph.
class AgentState(TypedDict):
    topic: str
    analysis: str
    feedback: list
    revision_count: int

# --- 2. Define the Graph Nodes (The "Workers") ---
def generator_node(state: AgentState):
    """
    Generates the financial analysis (or revises it based on feedback).
    """
    print(f"\n--- Turn {state['revision_count'] + 1}: GENERATING ANALYSIS ---")
    topic = state['topic']
    feedback = state.get('feedback', [])

    if feedback:
        print("  -> Revising based on feedback...")
        prompt = f"Revise the following analysis based on the feedback. Produce an improved, more balanced version.\n\nFeedback: {feedback[-1]}\n\nAnalysis to Revise: {state['analysis']}"
    else:
        print("  -> Generating initial draft...")
        prompt = f"Generate a concise financial analysis for {topic}. Cover its primary strengths, weaknesses, and a concluding outlook."

    analysis = llm.invoke(prompt).content
    return {"analysis": analysis, "revision_count": state['revision_count'] + 1}

def evaluator_node(state: AgentState):
    """Evaluates the analysis and provides feedback or approval."""
    print("--- EVALUATING ANALYSIS ---")
    analysis = state['analysis']
    prompt = f"You are a Senior Investment Manager. Evaluate this analysis. Is it detailed, balanced, and actionable? If it is good enough to present, respond with only 'OK'. Otherwise, provide one sentence of constructive feedback.\n\nAnalysis:\n{analysis}"
    evaluation = llm.invoke(prompt).content

    if "OK" in evaluation:
        print("  -> Evaluation: Analysis Approved.")
        return {"feedback": []}  # Empty feedback list signals completion
    else:
        print(f"  -> Evaluation: Feedback provided -> '{evaluation.strip()}'")
        return {"feedback": state['feedback'] + [evaluation.strip()]}

# --- 3. Define the Conditional Edge (The "Manager") ---
def should_continue(state: AgentState):
    """Determines whether to loop back for revision or to end."""
    if state['feedback']:
        if state['revision_count'] >= 2: # Set a limit to avoid infinite loops
            print("--- Max revisions reached. Ending loop. ---")
            return "end"
        return "continue"
    else:
        print("--- No feedback. Finalizing analysis. ---")
        return "end"

# --- 4. Build and Run the Graph ---
workflow = StateGraph(AgentState)
workflow.add_node("generator", generator_node)
workflow.add_node("evaluator", evaluator_node)

workflow.set_entry_point("generator")
workflow.add_edge("generator", "evaluator")
workflow.add_conditional_edges("evaluator", should_continue, {"continue": "generator", "end": END})

app = workflow.compile()

# Let's run the loop!
initial_state = {"topic": "Tesla (TSLA)", "feedback": [], "revision_count": 0}
final_state = app.invoke(initial_state)

print("\n\n--- ✅ FINAL, REFINED ANALYSIS ---")
print(final_state['analysis'])

--- ⚙️ Pattern 3: Evaluator-Optimizer Loop with LangGraph ---

--- Turn 1: GENERATING ANALYSIS ---
  -> Generating initial draft...
--- EVALUATING ANALYSIS ---
  -> Evaluation: Analysis Approved.
--- No feedback. Finalizing analysis. ---


--- ✅ FINAL, REFINED ANALYSIS ---
Here's a concise financial analysis for Tesla (TSLA):

**Tesla (TSLA) - Concise Financial Analysis**

**Strengths:**

1.  **Market Leadership & Brand:** Dominant position in the electric vehicle (EV) market, fueled by strong global brand recognition, an integrated Supercharger network, and a loyal customer base.
2.  **Technological Prowess:** Advanced battery technology, proprietary software (Autopilot/FSD potential), and efficient manufacturing processes (Gigafactories) provide a competitive edge.
3.  **Diversified Growth Avenues:** Growing energy generation and storage division (Powerwall, Megapack) and ambitious ventures into AI/robotics offer significant long-term diversification and revenue potential.

**Weaknes

# Build the Agents

In [7]:
# Prompt chaining for this Agent which takes news and distill it into a
# clear sentiment: positive or negative
# sentiment_chain | entities_chain | summary_chain

# (Ingest News → Classify Sentiment → Extract Entities → Summarize)
# main tool for this agent:
class NewsSentimentAnalystAgent:

    def analyze_news(self, article_text):

        # chaining logic goes here
        sentiment = sentiment_chain.invoke({"article": article_text})
        entities = entities_chain.invoke({"article": article_text})
        summary = summary_chain.invoke({"sentiment": sentiment, "article": article_text})

        return {
            "sentiment": sentiment, "entities": entities, "summary": summary
            }

In [8]:
# The final prompt for the Manager Agent
final_report_prompt = """
You are a Chief Investment Strategist. Your task is to create a final investment thesis.

You have received the following inputs:

1.  **Quantitative Analysis:**
    {quantitative_report}

2.  **News & Sentiment Analysis:**
    {news_summary}

Synthesize these two distinct pieces of information into a coherent report. Your report must include:
- An Executive Summary
- Key Quantitative Highlights
- News & Sentiment Overview
- A Final Recommendation (Buy/Hold/Sell) with a 3-point justification.
"""